#### This notebook shows how to read the fastMRI dataset and apply some simple transformations to the data.

In [ ]:
# Testing if integration works

In [ ]:
%matplotlib inline

import h5py
import numpy as np
from matplotlib import pyplot as plt

The fastMRI dataset is distributed as a set of HDF5 files and can be read with the h5py package. Here, we show how to open a file from the multi-coil dataset. Each file corresponds to one MRI scan and contains the k-space data, ground truth and some meta data related to the scan.

In [ ]:
# file_name = 'multicoil_train/file1000167.h5'
# root = '/mnt/e/fastMRI/'
root = '/home/littlelinux/git/fastMRI/fastmri_examples/unet/results/'
# file_name = root + 'singlecoil_train/file1000005.h5'
# file_name = root + 'original/file1000017.h5'
file_name = root + 'reconstructions/file1000017.h5'
hf = h5py.File(file_name)

In [ ]:
print('Keys:', list(hf.keys()))
print('Attrs:', dict(hf.attrs))

In multi-coil MRIs, k-space has the following shape:
(number of slices, number of coils, height, width)

For single-coil MRIs, k-space has the following shape:
(number of slices, height, width)

MRIs are acquired as 3D volumes, the first dimension is the number of 2D slices.

In [ ]:
# volume_kspace = hf['kspace'][()]
volume_kspace = hf['reconstruction'][()]
print(volume_kspace.dtype)
print(volume_kspace.shape)

In [ ]:
plt.imshow(volume_kspace[20, 0, :, :],cmap='gray')

In [ ]:
from helper import *

file_list = [
    'file1000017.h5',
    'file1000126.h5',
    'file1000263.h5',
    'file1000323.h5',
    'file1000538.h5',
    'file1000810.h5',
    'file1000817.h5',
#     'file1000990.h5',
#     'file1001077.h5',
#     'file1001191.h5',
    'file1001499.h5',
]

n_total = 0
p_total = 0

for file in file_list:
    print(file)
    target_root = '/home/littlelinux/git/fastMRI/fastmri_examples/unet/results/original/'
    pred_root = '/home/littlelinux/git/fastMRI/fastmri_examples/unet/results/reconstructions/'
    h5e = h5py.File(pred_root + file)
    h5a = h5py.File(target_root + file)
    
    print('Keys:', list(h5a.keys()))
    print('Attrs:', dict(h5a.attrs))
    
    test_gen = h5e['reconstruction']
    target = h5a['reconstruction_rss']
    test_gen = test_gen[:, 0, :, :]

    n_total += nmse(test_gen, target)
    p_total += psnr(test_gen, target)
    h5e.close()
    h5a.close()

print(n_total/11)
print(p_total/11)

In [ ]:
slice_kspace = volume_kspace[20] # Choosing the 20-th slice of this volume

Let's see what the absolute value of k-space looks like:

In [ ]:
def show_coils(data, slice_nums, cmap=None):
    fig = plt.figure()
    for i, num in enumerate(slice_nums):
        plt.subplot(1, len(slice_nums), i + 1)
        plt.imshow(data[num], cmap=cmap)
        
def show_coil_single(data, cmap=None):
    fig = plt.figure()
#     for i, num in enumerate(slice_nums):
#         plt.subplot(1, len(slice_nums), i + 1)
    plt.imshow(data, cmap=cmap)

In [ ]:
# show_coils(np.log(np.abs(slice_kspace) + 1e-9), [0, 5, 10])  # This shows coils 0, 5 and 10
show_coil_single(np.log(np.abs(slice_kspace) + 1e-9))

The fastMRI repo contains some utlity functions to convert k-space into image space. These functions work on PyTorch Tensors. The to_tensor function can convert Numpy arrays to PyTorch Tensors.

In [ ]:
import fastmri
from fastmri.data import transforms as T

In [ ]:
slice_kspace2 = T.to_tensor(slice_kspace)      # Convert from numpy array to pytorch tensor
slice_image = fastmri.ifft2c(slice_kspace2)           # Apply Inverse Fourier Transform to get the complex image
slice_image_abs = fastmri.complex_abs(slice_image)   # Compute absolute value to get a real image

In [ ]:
# show_coils(slice_image_abs, [0, 5, 10], cmap='gray')
show_coil_single(slice_image_abs, cmap='gray')

As we can see, each coil in a multi-coil MRI scan focusses on a different region of the image. These coils can be combined into the full image using the Root-Sum-of-Squares (RSS) transform.

In [ ]:
slice_image_rss = fastmri.rss(slice_image_abs, dim=0)

In [ ]:
plt.imshow(np.abs(slice_image_rss.numpy()), cmap='gray')

So far, we have been looking at fully-sampled data. We can simulate under-sampled data by creating a mask and applying it to k-space.

In [ ]:
from fastmri.data.subsample import RandomMaskFunc
mask_func = RandomMaskFunc(center_fractions=[0.04], accelerations=[8])  # Create the mask function object

In [ ]:
masked_kspace, mask, _ = T.apply_mask(slice_kspace2, mask_func)   # Apply the mask to k-space

Let's see what the subsampled image looks like:

In [ ]:
sampled_image = fastmri.ifft2c(masked_kspace)           # Apply Inverse Fourier Transform to get the complex image
sampled_image_abs = fastmri.complex_abs(sampled_image)   # Compute absolute value to get a real image
sampled_image_rss = fastmri.rss(sampled_image_abs, dim=0)

In [ ]:
plt.imshow(np.abs(sampled_image_rss.numpy()), cmap='gray')